# Tutorial 2: Homología Persistente Avanzada

## Aplicaciones a Patrones Neuronales

**Autor:** MARK-126  
**Nivel:** Intermedio-Avanzado  
**Tiempo estimado:** 120-150 minutos

---

## Objetivos de Aprendizaje

1. ✅ Dominar diferentes tipos de filtraciones (Rips, Alpha, Čech)
2. ✅ Calcular distancias entre diagramas de persistencia
3. ✅ Aplicar TDA a spike trains neuronales
4. ✅ Optimizar cálculos para datasets grandes
5. ✅ Interpretar resultados en contexto neurobiológico

---

## 1. Repaso y Motivación

En el Tutorial 1 aprendimos los fundamentos de TDA. Ahora vamos más profundo:

### ¿Por qué necesitamos métodos avanzados?

En neurociencias nos encontramos con:
- **Datos masivos:** Miles de neuronas registradas simultáneamente
- **Alta dimensión:** Espacios de activación de 100+ dimensiones
- **Comparaciones:** Necesitamos cuantificar similitud entre estados
- **Temporalidad:** Datos dinámicos que evolucionan en el tiempo

### Preguntas que responderemos:

1. ¿Cómo comparar la topología de dos estados cerebrales?
2. ¿Qué filtración es mejor para datos neuronales?
3. ¿Cómo analizar patrones temporales de activación?
4. ¿Qué características topológicas son más informativas?

---

In [ ]:
# Importaciones
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# TDA avanzado
from ripser import ripser
from persim import plot_diagrams, bottleneck, sliced_wasserstein
import gudhi as gd
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Amplitude, PersistenceEntropy
from gtda.plotting import plot_diagram

# Procesamiento de datos
from scipy.spatial.distance import pdist, squareform, euclidean
from scipy.stats import poisson
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Visualización avanzada
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
np.random.seed(42)

print("✅ Bibliotecas importadas correctamente")
print(f"📦 GUDHI version: {gd.__version__}")

---

## 2. Tipos de Filtraciones

### 2.1 ¿Qué es una filtración?

Una **filtración** es una secuencia de complejos simpliciales anidados:
$$\emptyset = K_0 \subseteq K_1 \subseteq K_2 \subseteq \ldots \subseteq K_n = K$$

Cada tipo de filtración tiene **ventajas específicas** para diferentes tipos de datos.

### 2.2 Comparación de Filtraciones

| Filtración | Ventaja | Desventaja | Aplicación Neural |
|-----------|---------|------------|-------------------|
| **Vietoris-Rips** | Rápido, simple | Puede añadir simplejos espurios | Análisis de conectividad general |
| **Čech** | Teóricamente óptimo | Computacionalmente costoso | Estudios teóricos |
| **Alpha** | Geométricamente preciso | Solo para baja dimensión | Visualización de subredes |
| **Filtración de grafo** | Natural para redes | Requiere estructura de grafo | Conectomas cerebrales |

---

## 3. Implementación Práctica: Comparación de Filtraciones

In [ ]:
def compare_filtrations(points, max_dim=2):
    """
    Compara diferentes tipos de filtraciones en los mismos datos.
    
    Parameters:
    -----------
    points : numpy array
        Puntos a analizar (n_points x n_dimensions)
    max_dim : int
        Dimensión máxima de homología
    """
    results = {}
    
    # 1. Vietoris-Rips (usando ripser - rápido)
    print("🔄 Calculando Vietoris-Rips...")
    import time
    start = time.time()
    rips_result = ripser(points, maxdim=max_dim)
    rips_time = time.time() - start
    results['Vietoris-Rips'] = {
        'diagrams': rips_result['dgms'],
        'time': rips_time
    }
    
    # 2. Alpha complex (usando GUDHI)
    if points.shape[1] <= 3:  # Alpha solo funciona bien en 2D/3D
        print("🔄 Calculando Alpha complex...")
        start = time.time()
        alpha_complex = gd.AlphaComplex(points=points)
        simplex_tree = alpha_complex.create_simplex_tree()
        persistence = simplex_tree.persistence()
        
        # Convertir a formato de diagrama
        alpha_diagrams = [[] for _ in range(max_dim + 1)]
        for dim, (birth, death) in persistence:
            if dim <= max_dim:
                alpha_diagrams[dim].append([birth, death])
        
        alpha_diagrams = [np.array(d) if len(d) > 0 else np.array([]).reshape(0, 2) 
                         for d in alpha_diagrams]
        alpha_time = time.time() - start
        
        results['Alpha'] = {
            'diagrams': alpha_diagrams,
            'time': alpha_time
        }
    
    return results

# Generar datos de prueba: red neuronal sintética
print("🧠 Generando red neuronal sintética...\n")
n_neurons = 50
neural_data_2d = np.random.randn(n_neurons, 2) * 0.5
neural_data_2d[:20] += np.array([2, 0])  # Comunidad 1
neural_data_2d[20:35] += np.array([0, 2])  # Comunidad 2
# Comunidad 3 en el origen

# Comparar filtraciones
results = compare_filtrations(neural_data_2d, max_dim=1)

# Visualizar resultados
fig, axes = plt.subplots(1, len(results) + 1, figsize=(18, 5))

# Plot 1: Datos originales
axes[0].scatter(neural_data_2d[:, 0], neural_data_2d[:, 1], 
                c='blue', s=100, alpha=0.6, edgecolors='black')
axes[0].set_title('Red Neuronal\n(3 comunidades)', fontsize=12, fontweight='bold')
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)

# Plots de diagramas
for idx, (name, result) in enumerate(results.items(), start=1):
    plot_diagrams(result['diagrams'], ax=axes[idx])
    axes[idx].set_title(f'{name}\n(tiempo: {result["time"]:.3f}s)', 
                       fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Análisis completado")
print("\n📊 Tiempos de cómputo:")
for name, result in results.items():
    print(f"   • {name}: {result['time']:.4f} segundos")

### 💡 Observaciones

- **Vietoris-Rips** es generalmente más rápido
- **Alpha** es más preciso geométricamente pero limitado a 2D/3D
- Para datos neuronales de alta dimensión, **Rips es la elección práctica**

---

## 4. Distancias entre Diagramas de Persistencia

### 4.1 ¿Por qué necesitamos distancias?

Para comparar estados cerebrales, necesitamos **cuantificar diferencias** entre sus topologías.

### 4.2 Principales Métricas

#### A. Distancia de Bottleneck
$$d_B(D_1, D_2) = \inf_{\gamma} \sup_{p \in D_1} \|p - \gamma(p)\|_\infty$$

**Interpretación:** Peor caso de emparejamiento entre puntos
- **Robusta** a outliers
- Mide la **máxima diferencia** entre características

#### B. Distancia de Wasserstein
$$W_p(D_1, D_2) = \left(\inf_{\gamma} \sum_{p \in D_1} \|p - \gamma(p)\|^p\right)^{1/p}$$

**Interpretación:** Costo promedio de transporte
- **Sensible** a todas las características
- Mide **diferencia global**

#### C. Sliced Wasserstein
Aproximación rápida de Wasserstein mediante proyecciones 1D.

---

## 5. Aplicación: Comparación de Estados Cerebrales

In [ ]:
def generate_brain_state_realistic(state_type, n_neurons=100, noise=0.1):
    """
    Genera estados cerebrales sintéticos con propiedades realistas.
    
    Parameters:
    -----------
    state_type : str
        'sleep', 'wakeful', 'attention', 'memory'
    n_neurons : int
        Número de neuronas
    noise : float
        Nivel de ruido
    """
    if state_type == 'sleep':
        # Sueño: activación sincronizada, baja dimensionalidad
        base = np.random.randn(n_neurons, 1) @ np.random.randn(1, 5)
        data = base + np.random.randn(n_neurons, 5) * noise
        
    elif state_type == 'wakeful':
        # Vigilia: activación dispersa, alta dimensionalidad
        data = np.random.randn(n_neurons, 5) * 1.5
        
    elif state_type == 'attention':
        # Atención: subredes focales activas
        data = np.zeros((n_neurons, 5))
        # Subred atencional activa
        data[:n_neurons//3] = np.random.randn(n_neurons//3, 5) * 2.0
        # Resto con actividad basal
        data[n_neurons//3:] = np.random.randn(2*n_neurons//3, 5) * 0.3
        
    elif state_type == 'memory':
        # Memoria: patrones cíclicos (bucles de retroalimentación)
        theta = np.linspace(0, 4*np.pi, n_neurons)
        data = np.column_stack([
            np.cos(theta),
            np.sin(theta),
            np.cos(2*theta) * 0.5,
            np.sin(2*theta) * 0.5,
            np.random.randn(n_neurons) * noise
        ])
    
    return data

# Generar diferentes estados
print("🧠 Generando estados cerebrales sintéticos...\n")
states = {
    'Sueño': generate_brain_state_realistic('sleep', n_neurons=80),
    'Vigilia': generate_brain_state_realistic('wakeful', n_neurons=80),
    'Atención': generate_brain_state_realistic('attention', n_neurons=80),
    'Memoria': generate_brain_state_realistic('memory', n_neurons=80)
}

# Calcular persistencia para cada estado
print("⏳ Calculando homología persistente...\n")
diagrams = {}
for name, data in states.items():
    result = ripser(data, maxdim=1, thresh=3.0)
    diagrams[name] = result['dgms']
    print(f"✓ {name}: H₀={len(result['dgms'][0])}, H₁={len(result['dgms'][1])}")

# Calcular matriz de distancias
print("\n📏 Calculando distancias entre estados...\n")
state_names = list(states.keys())
n_states = len(state_names)

# Matrices para diferentes métricas
bottleneck_matrix = np.zeros((n_states, n_states))
wasserstein_matrix = np.zeros((n_states, n_states))

for i, name1 in enumerate(state_names):
    for j, name2 in enumerate(state_names):
        if i <= j:
            # Usar H₁ (ciclos) para comparación
            d1 = diagrams[name1][1]
            d2 = diagrams[name2][1]
            
            if len(d1) > 0 and len(d2) > 0:
                bottleneck_matrix[i, j] = bottleneck(d1, d2)
                bottleneck_matrix[j, i] = bottleneck_matrix[i, j]
                
                wasserstein_matrix[i, j] = sliced_wasserstein(d1, d2)
                wasserstein_matrix[j, i] = wasserstein_matrix[i, j]

# Visualizar matrices de distancia
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bottleneck
im1 = axes[0].imshow(bottleneck_matrix, cmap='YlOrRd', aspect='auto')
axes[0].set_xticks(range(n_states))
axes[0].set_yticks(range(n_states))
axes[0].set_xticklabels(state_names, rotation=45)
axes[0].set_yticklabels(state_names)
axes[0].set_title('Distancia de Bottleneck\n(H₁ - Ciclos)', fontsize=14, fontweight='bold')
plt.colorbar(im1, ax=axes[0])

# Agregar valores en las celdas
for i in range(n_states):
    for j in range(n_states):
        text = axes[0].text(j, i, f'{bottleneck_matrix[i, j]:.2f}',
                          ha="center", va="center", color="black", fontweight='bold')

# Wasserstein
im2 = axes[1].imshow(wasserstein_matrix, cmap='YlGnBu', aspect='auto')
axes[1].set_xticks(range(n_states))
axes[1].set_yticks(range(n_states))
axes[1].set_xticklabels(state_names, rotation=45)
axes[1].set_yticklabels(state_names)
axes[1].set_title('Distancia de Sliced Wasserstein\n(H₁ - Ciclos)', fontsize=14, fontweight='bold')
plt.colorbar(im2, ax=axes[1])

# Agregar valores
for i in range(n_states):
    for j in range(n_states):
        text = axes[1].text(j, i, f'{wasserstein_matrix[i, j]:.2f}',
                          ha="center", va="center", color="black", fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Análisis de distancias completado")
print("\n💡 Interpretación:")
print("   • Estados con distancia BAJA son topológicamente similares")
print("   • Estados con distancia ALTA tienen estructuras diferentes")
print(f"   • Par más similar: {state_names[np.unravel_index(np.argmin(bottleneck_matrix + np.eye(n_states)*999), bottleneck_matrix.shape)]}")

### 🔬 Análisis Neurobiológico

**Observaciones esperadas:**

1. **Sueño vs Vigilia:** Distancia alta (estructuras muy diferentes)
2. **Atención vs Memoria:** Ambos pueden tener ciclos (retroalimentación)
3. **Vigilia vs Atención:** Similar en estructura global, diferente en localización

**Significado de ciclos (H₁):**
- Sueño: Pocos ciclos (actividad sincronizada)
- Memoria: Muchos ciclos (bucles de retroalimentación)
- Vigilia: Ciclos distribuidos uniformemente

---

## 6. Aplicación Real: Análisis de Spike Trains

### 6.1 ¿Qué son los Spike Trains?

Los **spike trains** son secuencias de potenciales de acción (spikes) de neuronas:
```
Neurona 1: |-----|---------|---|-----|--------|
Neurona 2: |---|-----|----------|--|----------|
Neurona 3: |--------|---|-----|---------------|--|
           0   100   200   300   400   500   600 ms
```

### 6.2 Construcción del Espacio de Estados

Para aplicar TDA:
1. **Ventana deslizante:** Dividir en bins temporales
2. **Vector de activación:** Contar spikes por neurona en cada bin
3. **Espacio de estados:** Cada bin = punto en espacio de dimensión N (N = # neuronas)
4. **Analizar topología** de la trayectoria

---

In [ ]:
def generate_spike_trains(n_neurons=20, duration=1000, base_rate=5.0, 
                         correlation=0.3, pattern_type='random'):
    """
    Genera spike trains sintéticos con diferentes patrones.
    
    Parameters:
    -----------
    n_neurons : int
        Número de neuronas
    duration : int
        Duración en ms
    base_rate : float
        Tasa de disparo base (Hz)
    correlation : float
        Nivel de correlación entre neuronas
    pattern_type : str
        'random', 'synchronized', 'sequential'
    """
    spike_trains = np.zeros((n_neurons, duration))
    
    if pattern_type == 'random':
        # Actividad aleatoria independiente
        for i in range(n_neurons):
            spike_trains[i] = poisson.rvs(base_rate/1000, size=duration)
            
    elif pattern_type == 'synchronized':
        # Actividad sincronizada (todas disparan juntas)
        common_pattern = poisson.rvs(base_rate/1000, size=duration)
        for i in range(n_neurons):
            spike_trains[i] = common_pattern * (np.random.rand(duration) < 0.8)  # 80% sync
            
    elif pattern_type == 'sequential':
        # Actividad secuencial (onda de activación)
        for t in range(duration):
            active_neuron = (t // 20) % n_neurons  # Cambia cada 20ms
            spike_trains[active_neuron, t] = poisson.rvs(base_rate*3/1000)
    
    return spike_trains

def spike_trains_to_state_space(spike_trains, bin_size=50, stride=25):
    """
    Convierte spike trains a representación en espacio de estados.
    
    Parameters:
    -----------
    spike_trains : numpy array
        Matriz de spikes (n_neurons x time)
    bin_size : int
        Tamaño de ventana en ms
    stride : int
        Paso de la ventana deslizante
    """
    n_neurons, duration = spike_trains.shape
    n_bins = (duration - bin_size) // stride + 1
    
    state_space = np.zeros((n_bins, n_neurons))
    
    for i in range(n_bins):
        start = i * stride
        end = start + bin_size
        state_space[i] = np.sum(spike_trains[:, start:end], axis=1)
    
    return state_space

# Generar spike trains con diferentes patrones
print("🔥 Generando spike trains con diferentes patrones...\n")

patterns = ['random', 'synchronized', 'sequential']
spike_data = {}
state_spaces = {}

for pattern in patterns:
    spikes = generate_spike_trains(n_neurons=15, duration=1000, 
                                  pattern_type=pattern, base_rate=8.0)
    spike_data[pattern] = spikes
    state_spaces[pattern] = spike_trains_to_state_space(spikes, bin_size=50, stride=25)

# Visualizar spike trains
fig, axes = plt.subplots(3, 2, figsize=(18, 12))

for idx, pattern in enumerate(patterns):
    # Raster plot
    ax1 = axes[idx, 0]
    for neuron in range(spike_data[pattern].shape[0]):
        spike_times = np.where(spike_data[pattern][neuron] > 0)[0]
        ax1.scatter(spike_times, [neuron]*len(spike_times), 
                   c='black', s=1, marker='|')
    ax1.set_ylabel('Neurona #', fontsize=11)
    ax1.set_xlabel('Tiempo (ms)', fontsize=11)
    ax1.set_title(f'Spike Raster: {pattern.capitalize()}', 
                 fontsize=12, fontweight='bold')
    ax1.set_xlim([0, 1000])
    
    # Espacio de estados (proyección 2D con PCA)
    ax2 = axes[idx, 1]
    if state_spaces[pattern].shape[0] > 2:
        pca = PCA(n_components=2)
        reduced = pca.fit_transform(state_spaces[pattern])
        
        ax2.plot(reduced[:, 0], reduced[:, 1], 'o-', 
                alpha=0.6, linewidth=2, markersize=4)
        ax2.scatter(reduced[0, 0], reduced[0, 1], 
                   c='green', s=200, marker='o', zorder=5, 
                   edgecolors='black', linewidth=2, label='Inicio')
        ax2.scatter(reduced[-1, 0], reduced[-1, 1], 
                   c='red', s=200, marker='s', zorder=5, 
                   edgecolors='black', linewidth=2, label='Final')
        ax2.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', fontsize=11)
        ax2.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})', fontsize=11)
        ax2.set_title(f'Trayectoria en Espacio de Estados: {pattern.capitalize()}',
                     fontsize=12, fontweight='bold')
        ax2.legend(loc='best')
        ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualización completada")

### 🧠 Interpretación de Patrones

**Random (Aleatorio):**
- Spikes distribuidos uniformemente
- Trayectoria errática en espacio de estados
- Topología: pocas características persistentes

**Synchronized (Sincronizado):**
- Todas las neuronas disparan juntas
- Trayectoria lineal (baja dimensionalidad intrínseca)
- Topología: estructura simple, sin ciclos

**Sequential (Secuencial):**
- Activación en cascada
- Trayectoria **cíclica** (patrón repetitivo)
- Topología: **bucles detectables** (H₁ > 0)

---

In [ ]:
# Análisis topológico de spike trains
print("🔍 Analizando topología de patrones de disparo...\n")

spike_diagrams = {}

for pattern in patterns:
    print(f"⏳ Procesando patrón: {pattern}")
    result = ripser(state_spaces[pattern], maxdim=1, thresh=15.0)
    spike_diagrams[pattern] = result['dgms']
    print(f"   H₀: {len(result['dgms'][0])} componentes")
    print(f"   H₁: {len(result['dgms'][1])} ciclos\n")

# Visualizar diagramas de persistencia
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, pattern in enumerate(patterns):
    plot_diagrams(spike_diagrams[pattern], ax=axes[idx])
    axes[idx].set_title(f'Persistencia: {pattern.capitalize()}\n' + 
                       f'H₁ = {len(spike_diagrams[pattern][1])} ciclos',
                       fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Hallazgos Clave:")
print("   • El patrón SECUENCIAL debe mostrar ciclos robustos (H₁ alto)")
print("   • El patrón SINCRONIZADO debe tener estructura simple (H₁ bajo)")
print("   • El patrón ALEATORIO debe tener características cerca de la diagonal")
print("\n✅ TDA detecta exitosamente patrones temporales en actividad neuronal!")

---

## 7. Características Topológicas para Machine Learning

### 7.1 Vectorización de Diagramas

Para usar TDA en ML, necesitamos convertir diagramas a vectores de características:

1. **Amplitud de Persistencia:** $\max(death - birth)$
2. **Entropía de Persistencia:** $-\sum p_i \log(p_i)$ donde $p_i = \frac{L_i}{\sum L_j}$
3. **Número de Betti:** Contar características en umbrales específicos
4. **Estadísticas de lifetime:** Media, mediana, desviación

### 7.2 Aplicación: Clasificación de Estados

---

In [ ]:
def extract_topological_features(diagram, dim=1):
    """
    Extrae características escalares de un diagrama de persistencia.
    
    Parameters:
    -----------
    diagram : numpy array
        Diagrama de persistencia (n_points x 2)
    dim : int
        Dimensión homológica
    """
    features = {}
    
    if len(diagram[dim]) == 0:
        return {'n_features': 0, 'max_persistence': 0, 
                'mean_persistence': 0, 'entropy': 0}
    
    # Filtrar infinitos
    dgm = diagram[dim][np.isfinite(diagram[dim][:, 1])]
    
    if len(dgm) == 0:
        return {'n_features': 0, 'max_persistence': 0, 
                'mean_persistence': 0, 'entropy': 0}
    
    # Lifetimes (persistencias)
    lifetimes = dgm[:, 1] - dgm[:, 0]
    
    # Características
    features['n_features'] = len(dgm)
    features['max_persistence'] = np.max(lifetimes)
    features['mean_persistence'] = np.mean(lifetimes)
    features['std_persistence'] = np.std(lifetimes)
    features['total_persistence'] = np.sum(lifetimes)
    
    # Entropía de persistencia
    if np.sum(lifetimes) > 0:
        probs = lifetimes / np.sum(lifetimes)
        entropy = -np.sum(probs * np.log(probs + 1e-10))
        features['entropy'] = entropy
    else:
        features['entropy'] = 0
    
    return features

# Extraer características de todos los patrones
print("📊 Extrayendo características topológicas...\n")

feature_summary = []

for pattern in patterns:
    feats = extract_topological_features(spike_diagrams[pattern], dim=1)
    feats['pattern'] = pattern
    feature_summary.append(feats)

# Crear DataFrame
df_features = pd.DataFrame(feature_summary)
df_features = df_features[['pattern', 'n_features', 'max_persistence', 
                           'mean_persistence', 'entropy']]

print("Características Topológicas (H₁ - Ciclos):\n")
print(df_features.to_string(index=False))
print("\n")

# Visualizar características
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

features_to_plot = ['n_features', 'max_persistence', 'mean_persistence', 'entropy']
titles = ['Número de Ciclos', 'Máxima Persistencia', 
          'Persistencia Promedio', 'Entropía de Persistencia']
colors = ['#e74c3c', '#3498db', '#2ecc71']

for idx, (feat, title) in enumerate(zip(features_to_plot, titles)):
    ax = axes[idx // 2, idx % 2]
    values = [df_features[df_features['pattern'] == p][feat].values[0] for p in patterns]
    bars = ax.bar(patterns, values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax.set_ylabel(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Patrón', fontsize=11)
    ax.set_title(f'{title} por Patrón de Activación', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Agregar valores en las barras
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ Extracción de características completada")
print("\n💡 Observa cómo cada patrón tiene una 'firma topológica' única!")

### 🎯 Firmas Topológicas

Cada tipo de actividad neuronal tiene características topológicas distintivas:

| Patrón | N° Ciclos | Max Persist | Entropía | Interpretación |
|--------|-----------|-------------|----------|----------------|
| **Random** | Bajo | Bajo | Baja | Sin estructura | 
| **Synchronized** | Muy bajo | Bajo | Muy baja | Colapso dimensional |
| **Sequential** | Alto | Alto | Alta | Estructura cíclica robusta |

**Aplicación práctica:** Estas características pueden usarse para:
- Clasificar estados cerebrales
- Detectar patrones anormales
- Predecir transiciones de estado

---

## 8. Optimización para Grandes Datasets

### 8.1 Desafíos Computacionales

En neurociencias reales:
- **Miles de neuronas:** N = 1000-10000
- **Alta dimensionalidad:** Espacios de 100+ dimensiones
- **Datos temporales:** Miles de time bins

### 8.2 Estrategias de Optimización

1. **Subsampling:** Reducir número de puntos
2. **Landmark selection:** Elegir puntos representativos
3. **Threshold early:** Limitar radio máximo
4. **Dimensión reducida:** PCA/UMAP antes de TDA
5. **Sparse distance matrix:** Solo distancias < threshold

---

In [ ]:
def benchmark_tda_methods(n_points_list, n_dimensions=10):
    """
    Compara tiempos de cómputo para diferentes tamaños de datos.
    """
    import time
    
    results = {'n_points': [], 'time': [], 'method': []}
    
    for n_points in n_points_list:
        print(f"\n📊 Benchmarking con {n_points} puntos...")
        data = np.random.randn(n_points, n_dimensions)
        
        # Método 1: Ripser estándar
        start = time.time()
        result = ripser(data, maxdim=1, thresh=2.0)
        elapsed = time.time() - start
        results['n_points'].append(n_points)
        results['time'].append(elapsed)
        results['method'].append('Ripser (full)')
        print(f"   Ripser (full): {elapsed:.3f}s")
        
        # Método 2: Con reducción dimensional (PCA)
        start = time.time()
        pca = PCA(n_components=min(5, n_dimensions))
        data_reduced = pca.fit_transform(data)
        result = ripser(data_reduced, maxdim=1, thresh=2.0)
        elapsed = time.time() - start
        results['n_points'].append(n_points)
        results['time'].append(elapsed)
        results['method'].append('PCA + Ripser')
        print(f"   PCA + Ripser: {elapsed:.3f}s")
        
        # Método 3: Subsampling
        if n_points > 100:
            start = time.time()
            indices = np.random.choice(n_points, size=min(100, n_points), replace=False)
            data_sub = data[indices]
            result = ripser(data_sub, maxdim=1, thresh=2.0)
            elapsed = time.time() - start
            results['n_points'].append(n_points)
            results['time'].append(elapsed)
            results['method'].append('Subsampling (100)')
            print(f"   Subsampling: {elapsed:.3f}s")
    
    return pd.DataFrame(results)

# Ejecutar benchmark
print("⏱️ Ejecutando benchmark de métodos TDA...")
n_points_to_test = [50, 100, 200, 500]
benchmark_df = benchmark_tda_methods(n_points_to_test, n_dimensions=10)

# Visualizar resultados
fig, ax = plt.subplots(figsize=(12, 6))

for method in benchmark_df['method'].unique():
    data = benchmark_df[benchmark_df['method'] == method]
    ax.plot(data['n_points'], data['time'], 'o-', 
            linewidth=2, markersize=8, label=method)

ax.set_xlabel('Número de Puntos', fontsize=12, fontweight='bold')
ax.set_ylabel('Tiempo de Cómputo (s)', fontsize=12, fontweight='bold')
ax.set_title('Comparación de Eficiencia: Métodos TDA', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
plt.tight_layout()
plt.show()

print("\n✅ Benchmark completado")
print("\n💡 Recomendaciones:")
print("   • Para N < 200: Usa Ripser directamente")
print("   • Para N > 200: Considera PCA o subsampling")
print("   • Para dimensión alta: PCA a 5-10 dimensiones")
print("   • Siempre usa threshold adecuado para tu escala de datos")

---

## 9. Ejercicios Avanzados

### Ejercicio 1: Análisis de tus propios spike trains

Genera spike trains con parámetros personalizados y analiza su topología:

```python
# Crea un patrón intermitente (alterna entre sincronizado y aleatorio)
# Analiza cómo cambia la topología en el tiempo
```

---

In [ ]:
# Espacio para Ejercicio 1


### Ejercicio 2: Clasificador topológico

Usa características topológicas para entrenar un clasificador:

```python
from sklearn.ensemble import RandomForestClassifier

# 1. Genera múltiples ejemplos de cada patrón
# 2. Extrae características topológicas
# 3. Entrena un clasificador
# 4. Evalúa precisión
```

---

In [ ]:
# Espacio para Ejercicio 2


### Ejercicio 3: Análisis temporal

Estudia cómo evoluciona la topología durante una transición de estados:

```python
# Genera datos que transicionan de 'synchronized' a 'random'
# Calcula topología en ventanas deslizantes
# Visualiza la evolución de números de Betti
```

---

In [ ]:
# Espacio para Ejercicio 3


## 10. Resumen y Conclusiones

### ✅ Lo que dominamos:

1. **Filtraciones:** Diferentes métodos (Rips, Alpha, Čech) y cuándo usar cada uno
2. **Distancias:** Bottleneck y Wasserstein para comparar diagramas
3. **Spike trains:** Conversión a espacio de estados y análisis topológico
4. **Características:** Vectorización para machine learning
5. **Optimización:** Estrategias para datos masivos

### 🔑 Mensajes Clave:

- **TDA captura patrones temporales** en actividad neuronal
- **Ciclos (H₁)** revelan retroalimentación y patrones repetitivos
- **Firmas topológicas** son distintivas para cada tipo de actividad
- **Distancias entre diagramas** cuantifican similitud entre estados
- **Optimización es crucial** para aplicaciones reales

### 🧠 Impacto en Neurociencias:

TDA proporciona:
- **Descripción invariante** de patrones neuronales
- **Detección robusta** de estructuras funcionales
- **Comparación cuantitativa** entre condiciones
- **Base para biomarcadores** topológicos

---

## 11. Próximo Tutorial

En el **Tutorial 3: Conectividad Cerebral**, exploraremos:

- Análisis de conectomas (matrices de conectividad)
- TDA aplicado a redes funcionales vs estructurales
- Detección de comunidades topológicas
- Análisis de grafos cerebrales ponderados
- Estudios de caso con datos fMRI reales

---

## 12. Referencias

### Papers Clave:
1. Giusti et al. (2015). "Clique topology reveals intrinsic structure in neural correlations". *PNAS*
2. Petri et al. (2014). "Homological scaffolds of brain functional networks". *Journal of the Royal Society Interface*
3. Curto (2017). "What can topology tell us about the neural code?". *Bulletin of the AMS*
4. Sizemore et al. (2019). "Cliques and cavities in the human connectome". *Journal of Computational Neuroscience*

### Software:
- [Ripser](https://ripser.scikit-tda.org/)
- [GUDHI](https://gudhi.inria.fr/)
- [Giotto-TDA](https://giotto-ai.github.io/gtda-docs/)
- [Persim](https://persim.scikit-tda.org/)

---

**Autor:** MARK-126  
**Última actualización:** 2025-01-13  
**Licencia:** MIT